In [ ]:
# TODO
# publish script
# backup plans in case some tx doesnt go
# fund all relevant accounts
# include w3 gas price strategy?
# update ethpm directory

In [51]:
#### Raise your hands if you've ever used a package manager
#### Raise your hands if you've ever copy/pasted an ABI
#### Raise your hands if you've ever copy/pasted a contract address

In [ ]:
# The first rule of EthPM

In [ ]:
# Only install packages from **TRUSTED** registries.

![title](multisig/1.png)

# mission: save grandma

# Setup  <--
### Deploy "SaveGrandma" multisig
### Package up "SaveGrandma"
### Deploy new package registry & tie to ENS
### Publish "SaveGrandma" package
### Coop: Fund approve tx "SaveGrandma"
### Dr. : Approve "SaveGrandma"
### Save Grandma

In [2]:
from web3.auto.infura import w3
from web3.middleware import construct_sign_and_send_raw_middleware
from script import DR_PK, DR_ADDRESS, COOP_ADDRESS, coop_w3, GMA_ADDRESS, print_balances
from ens.utils import raw_name_to_hash
from ethpm.tools import builder
from ethpm.utils.chains import create_latest_block_uri
from eth_utils import to_canonical_address, to_hex
from ethpm.backends.ipfs import InfuraIPFSBackend

In [31]:
# Create the tx-signing middleware
signing_middleware = construct_sign_and_send_raw_middleware(DR_PK)

# Add middleware to target w3 instance
w3.middleware_onion.add(signing_middleware)

# Set default account to Address associated with middleware private key
w3.eth.defaultAccount = DR_ADDRESS

# Enable PM api (temporary)
w3.enable_unstable_package_management_api()

![title](multisig/4.png)

In [49]:
dir(w3.pm)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_validate_set_ens',
 '_validate_set_registry',
 'attach',
 'deploy_and_set_registry',
 'get_all_package_names',
 'get_all_package_releases',
 'get_package',
 'get_package_count',
 'get_package_from_manifest',
 'get_package_from_uri',
 'get_release_count',
 'get_release_data',
 'get_release_id',
 'get_release_id_data',
 'registry',
 'release_package',
 'set_registry',
 'web3']

![title](multisig/2.png)

In [41]:
w3.pm.registry

In [42]:
w3.pm.set_registry("snakecharmers.eth")

In [43]:
# ethpm.explorer
w3.pm.registry.address

'0x6b5DA3cA4286Baa7fBaf64EEEE1834C7d430B729'

##### Setup
# Deploy "SaveGrandma" multisig  <--
##### Package up "SaveGrandma"
##### Deploy new package registry & tie to ENS
##### Publish "SaveGrandma" package
##### Coop: Fund approve tx "SaveGrandma"
##### Dr. : Approve "SaveGrandma"
##### Save Grandma

![title](multisig/3.png)

In [44]:
multisig_pkg = w3.pm.get_package("multisig", "1.0.1")
print(type(multisig_pkg))

In [46]:
w3.pm.get_release_data("multisig", "1.0.1")

('multisig', '1.0.1', 'ipfs://QmdFykVygYNmGrLM6VUQGf9K2r2zjzeanoY32nh7c2NAwu')

In [47]:
# Show it in the explorer
multisig_pkg.uri

'ipfs://QmdFykVygYNmGrLM6VUQGf9K2r2zjzeanoY32nh7c2NAwu'

In [ ]:
multisig_factory = multisig_pkg.get_contract_factory("MultiSigWallet")

In [ ]:
tx_hash = multisig_factory.constructor([DR_ADDRESS, COOP_ADDRESS], 2).transact()
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [ ]:
# 0xF81c6b404f9b94d230b7c3Db85B6960d835B04dD
tx_receipt.contractAddress

##### Setup
##### Deploy "SaveGrandma" multisig
# Package up "SaveGrandma"  <--
##### Deploy new package registry & tie to ENS
##### Publish "SaveGrandma" package
##### Coop: Fund approve tx "SaveGrandma"
##### Dr. : Approve "SaveGrandma"
##### Save Grandma

![title](multisig/5.png)

In [ ]:
new_manifest = builder.build(
    multisig_pkg.manifest,
    builder.version("1.0.2"),
    builder.deployment(
        block_uri = create_latest_block_uri(w3, from_blocks_ago=0),
        contract_instance = 'SaveGrandma',
        contract_type = 'MultiSigWallet',
        address = to_canonical_address(tx_receipt.contractAddress),
        transaction = to_hex(tx_hash),
        block = to_hex(tx_receipt.blockHash),
    ),
    builder.validate(),
    builder.pin_to_ipfs(backend=InfuraIPFSBackend(), prettify=False),
)

In [ ]:
# Check it out on explorer
new_manifest

In [ ]:
# grab ENS pkg from snakecharmers.eth
ens_pkg = w3.pm.get_package("ens", "1.0.2")

##### Setup
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma" 
# Deploy new package registry & tie to ENS  <--
##### Publish "SaveGrandma" package
##### Coop: Fund approve tx "SaveGrandma"
##### Dr. : Approve "SaveGrandma"
##### Save Grandma

In [ ]:
# 0x46D533f1bdcD034785dbf7AcFca723c55B15b0b5
w3.pm.deploy_and_set_registry()

In [ ]:
print("Registry address: ", w3.pm.registry.address)
print("Registry owner:   ", w3.pm.registry.owner())

In [ ]:
# Tie registry to ens --- you'd never say check out this cool website :ip_addr:
ens = ens_pkg.deployments.get_instance("ENS")
resolver = ens_pkg.deployments.get_instance("PublicResolver")

In [ ]:
domain = raw_name_to_hash("drmantistoboggan.eth")

In [50]:
resolver.functions.addr(domain).call()

NameError: name 'resolver' is not defined

In [ ]:
txhash_ens = resolver.functions.setAddr(domain, w3.pm.registry.address).transact()
w3.eth.waitForTransactionReceipt(txhash_ens)

In [50]:
resolver.functions.addr(domain).call()

NameError: name 'resolver' is not defined

Future exception was never retrieved
future: <Future finished exception=ConnectionClosed('WebSocket connection is closed: code = 1006 (connection closed abnormally [internal]), no reason',)>
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 1006 (connection closed abnormally [internal]), no reason


##### Setup
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma" 
##### Deploy new package registry & tie to ENS
# Publish "SaveGrandma" package  <--
##### (Coop) Fund & approve tx "SaveGrandma"
##### (Dr.) Approve "SaveGrandma"
##### Save Grandma

In [ ]:
# Copy & paste new IPFS here - Returns release id
w3.pm.release_package("multisig", "1.0.2", "ipfs://QmbjeDnUqoXPHStCQaMRzyZGRGXsmaDgyg9iUMafLp6iTC")

In [ ]:
# ethpm.explorer
w3.pm.get_release_data("multisig", "1.0.2")

##### Setup
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma" 
##### Deploy new package registry  & tie to ENS
##### Publish "SaveGrandma" package 
# (Coop) Fund & approve tx "SaveGrandma"  <--
##### (Dr.) Approve "SaveGrandma"
##### Save Grandma

In [3]:
# coop_w3 already setup with auto-signing
coop_w3.pm.set_registry('drmantistoboggan.eth')

In [4]:
coop_w3.pm.registry.address

'0x46D533f1bdcD034785dbf7AcFca723c55B15b0b5'

In [5]:
coop_multisig_pkg = coop_w3.pm.get_package("multisig", "1.0.2")

In [6]:
save_grandma = coop_multisig_pkg.deployments.get_instance("SaveGrandma")

In [12]:
# Fund multisig
# Submit tx proposal
tx1_hash = save_grandma.fallback.transact({'value': coop_w3.toWei("0.01", "ether")})
tx1_receipt = coop_w3.eth.waitForTransactionReceipt(tx1_hash)


TypeError: 'NoneType' object is not subscriptable

In [16]:
tx2_hash = save_grandma.functions.submitTransaction(GMA_ADDRESS, coop_w3.toWei("0.005", "ether"), b'0x').transact()
tx2_receipt = coop_w3.eth.waitForTransactionReceipt(tx2_hash)
# set tx_id

TimeExhausted: Transaction b'D\xac\x97\x8d\xe1\xc5\x83\x1a\xbf~ \x9f\x06\\@\xb1$\x97\xee1\xb0\x1b\xd0\x16\x1d\xa1\xef_}\xc3\xaae' is not in the chain, after 120 seconds

In [ ]:
tx3_hash = save_grandma.functions.confirmTransaction(tx_id).transact()
coop_w3.eth.waitForTransactionReceipt(tx3_hash)

In [13]:
print_balances(coop_w3, save_grandma.address)

Grandma:  0
Multisig:  10000000000000000
Coop:  31041075188335019


In [22]:
coop_multisig.functions.getTransactionIds(0, 1, True, True).call()

[0]

In [28]:
coop_multisig.functions.getConfirmationCount(0).call()

1

In [38]:
print_balances(coop_w3, coop_multisig.address)

Grandma:  5000000000000000
Multisig:  5000000000000000
Coop:  30620559188335019


##### Setup
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma" 
##### Deploy new package registry & tie to ENS
##### Publish "SaveGrandma" package 
##### (Coop) Fund & approve tx "SaveGrandma" 
# (Dr.) Approve "SaveGrandma"  <--
##### Save Grandma

In [34]:
multisig_pkg = w3.pm.get_package("multisig", "1.0.2")

In [35]:
multisig = multisig_pkg.deployments.get_instance("SaveGrandma")

In [36]:
multisig.functions.confirmTransaction(0).transact()

HexBytes('0x61a7847b635e573a2ae5f7a5e53083d691dccca258f69c3a195ac7010bfcacd4')

# Grandma lives!

In [38]:
print_balances(coop_w3, coop_multisig.address)

Grandma:  5000000000000000
Multisig:  5000000000000000
Coop:  30620559188335019
